In [34]:
import numpy as np
import pandas as pd 

import os

# folder_path = r'..\My-CNN-projects\dataset\\'

print(os.listdir(r'C:\Users\bashah\OneDrive - Ncell Axiata Limited\ncell\Development\My-CNN-projects\dataset'))



['Face_expression']


In [36]:
import tensorflow as tf 

In [38]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Dense, Activation, Dropout, Flatten

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt 

In [40]:
filename = r'C:\Users\bashah\OneDrive - Ncell Axiata Limited\ncell\Development\My-CNN-projects\dataset\Face_expression\Dec2023.csv'
label_map = ['Anger',
             'Disgust',
             'Fear',
             'Happy',
             'Sad',
             'Surprise',
             'Neutral']
names=['emotion','pixels','usage']


In [41]:
df=pd.read_csv(r'C:\Users\bashah\OneDrive - Ncell Axiata Limited\ncell\Development\My-CNN-projects\dataset\Face_expression\Dec2023.csv', names=names, na_filter=False)


In [45]:
# im=df['pixel']
df.head(10)

,emotion,pixels,usage
0,emotion,pixels,Usage
1,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
2,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
3,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
4,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
5,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training
6,2,55 55 55 55 55 54 60 68 54 85 151 163 170 179 ...,Training
7,4,20 17 19 21 25 38 42 42 46 54 56 62 63 66 82 1...,Training
8,3,77 78 79 79 78 75 60 55 47 48 58 73 77 79 57 5...,Training
9,3,85 84 90 121 101 102 133 153 153 169 177 189 1...,Training


In [47]:
im = df['pixels']

In [50]:
def getData(filename):

    Y = []
    X = []

    first = True
    for line in open(filename):
        if first:
            first = False
        else:
            row = line.split(',')
            Y.append(int(row[0]))
            X.append([int(p) for p in row[1].split()])
            
    X, Y = np.array(X) /255.0, np.array(Y)
    return X, Y